## How to use

Provide a time-domain data series, saved as a xlsx file. The format of the matrix is:

1st column: Time axis
Remaining columns: Dependant variable data

Specify an intended interval for the data. 

The program will erase all samples which are not in line with the interval.

Intended use: FEM results (Comsol) in which convergence issues populate the data with extraneous samples.

In [104]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.decomposition import PCA as RandomizedPCA
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
import pickle
from PIL import Image
from pathlib import Path 

In [105]:
# datafile = 'G:\My Drive'
datafile = "I:\\My Drive\\USGW Research\\COMSOL Models\\Pipe 4\\Pipe 4\\d33 Displacements\\Viscoelastic\\Undamaged Attenuation.xlsx"

In [106]:
sheetname = 'Undamaged Attenuation'
data = pd.read_excel(datafile, sheetname) 
data = data.set_index('Time (ms)')
data = data.transpose()

data

Time (ms),0.000,0.005,0.010,0.015,0.020,0.025,0.030,0.035,0.040,0.045,...,4.955,4.960,4.965,4.970,4.975,4.980,4.985,4.990,4.995,5.000
"Radial Displacement A1 (mm), Point: (21, ro, 0)",-1.874100e-49,-8.174000e-06,-1.579400e-04,-5.529900e-04,-1.203900e-03,-2.021700e-03,-2.810300e-03,-3.303700e-03,-3.208400e-03,-2.271600e-03,...,6.114100e-06,6.612800e-06,7.041200e-06,0.000007,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008
"Radial Displacement A2 (mm), Point: (21, 0, ro)",-1.876300e-49,-8.174000e-06,-1.579300e-04,-5.528300e-04,-1.203100e-03,-2.020000e-03,-2.807700e-03,-3.300300e-03,-3.204700e-03,-2.268700e-03,...,6.772200e-06,6.883500e-06,6.923700e-06,0.000007,0.000007,0.000007,0.000006,0.000006,0.000006,0.000005
"Radial Displacement A3 (mm), Point: (21, -ro, 0)",-1.873600e-49,-8.175500e-06,-1.579600e-04,-5.530600e-04,-1.204000e-03,-2.022300e-03,-2.811400e-03,-3.304700e-03,-3.208500e-03,-2.269300e-03,...,4.534700e-06,4.973300e-06,5.359800e-06,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006
"Radial Displacement A4 (mm), Point: (21, 0, -ro)",-1.873600e-49,-8.176400e-06,-1.579800e-04,-5.531600e-04,-1.204300e-03,-2.022800e-03,-2.812200e-03,-3.305800e-03,-3.209500e-03,-2.269900e-03,...,4.173300e-06,4.977900e-06,5.728700e-06,0.000006,0.000007,0.000008,0.000008,0.000008,0.000009,0.000009
"Radial Displacement A1.5 (mm), Point: (21, 0.5*ro*sqrt(2), 0.5*ro*sqrt(2))",-1.846400e-49,-8.104300e-06,-1.566700e-04,-5.498700e-04,-1.199300e-03,-2.016300e-03,-2.804400e-03,-3.297700e-03,-3.203100e-03,-2.269400e-03,...,3.737400e-05,4.000600e-05,4.220900e-05,0.000044,0.000045,0.000046,0.000046,0.000046,0.000045,0.000044
"Radial Displacement A2.5 (mm), Point: (21, -0.5*ro*sqrt(2), 0.5*ro*sqrt(2))",-1.845400e-49,-8.120300e-06,-1.569800e-04,-5.509900e-04,-1.201600e-03,-2.019800e-03,-2.808900e-03,-3.302500e-03,-3.207400e-03,-2.271900e-03,...,3.968600e-05,4.325600e-05,4.637300e-05,0.000049,0.000051,0.000053,0.000054,0.000054,0.000054,0.000053
"Radial Displacement A3.5 (mm), Point: (21, -0.5*ro*sqrt(2), -0.5*ro*sqrt(2))",-1.849500e-49,-8.023000e-06,-1.550400e-04,-5.431900e-04,-1.183400e-03,-1.988400e-03,-2.765700e-03,-3.254000e-03,-3.164900e-03,-2.250500e-03,...,2.995500e-05,3.351100e-05,3.671600e-05,0.000040,0.000042,0.000044,0.000045,0.000046,0.000047,0.000047
"Radial Displacement A4.5 (mm), Point: (21, 0.5*ro*sqrt(2), -0.5*ro*sqrt(2))",-1.849400e-49,-8.046800e-06,-1.555000e-04,-5.448000e-04,-1.186600e-03,-1.993300e-03,-2.771500e-03,-3.259400e-03,-3.168000e-03,-2.249400e-03,...,2.552100e-05,2.822200e-05,3.063300e-05,0.000033,0.000034,0.000036,0.000037,0.000037,0.000038,0.000037
"Radial Displacement A'1 (mm), Point: (420, ro, 0)",2.929000e-78,-7.063200e-37,3.367100e-35,4.284900e-33,1.547800e-31,3.213500e-30,4.708100e-29,5.431800e-28,5.277900e-27,4.499900e-26,...,2.741200e-06,3.204900e-06,3.623700e-06,0.000004,0.000004,0.000005,0.000005,0.000005,0.000005,0.000005
"Radial Displacement A'2 (mm), Point: (420, 0, ro)",2.985800e-78,2.993400e-36,2.342200e-34,8.863900e-33,2.161500e-31,3.801300e-30,5.172100e-29,5.767600e-28,5.514600e-27,4.664800e-26,...,4.833600e-06,5.862100e-06,6.815600e-06,0.000008,0.000008,0.000009,0.000010,0.000010,0.000011,0.000011


In [107]:
interval = 0.01

Algorithm:
Take the time series, and loop over every value. Mod the value by the interval. If the solution is not zero, erase it and its response column from the dataframe. Save the resulting dataframe as a csv file. 

In [108]:
t = [];
for i in data.columns:
    t.append(i)

In [109]:
time_samples = np.arange(0,data.columns[-1]+interval,interval)
time_samples = np.round(time_samples,2)
strict_data = data

In [110]:
for i in strict_data.columns:
    if i in time_samples:
        continue
    else:
        strict_data = strict_data.drop(i,1)
        
strict_data

C:\Users\MEH-3\AppData\Local\Temp/ipykernel_27372/1020963553.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  strict_data = strict_data.drop(i,1)


Time (ms),0.00,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,...,4.91,4.92,4.93,4.94,4.95,4.96,4.97,4.98,4.99,5.00
"Radial Displacement A1 (mm), Point: (21, ro, 0)",-1.874100e-49,-1.579400e-04,-1.203900e-03,-2.810300e-03,-3.208400e-03,-3.426700e-04,6.260900e-03,1.422300e-02,1.877900e-02,1.513300e-02,...,-6.082100e-07,1.104400e-06,2.742900e-06,4.243600e-06,5.549500e-06,6.612800e-06,0.000007,0.000008,0.000008,0.000008
"Radial Displacement A2 (mm), Point: (21, 0, ro)",-1.876300e-49,-1.579300e-04,-1.203100e-03,-2.807700e-03,-3.204700e-03,-3.417500e-04,6.253500e-03,1.420500e-02,1.875400e-02,1.511300e-02,...,2.849100e-06,4.114000e-06,5.183100e-06,6.017700e-06,6.590000e-06,6.883500e-06,0.000007,0.000007,0.000006,0.000005
"Radial Displacement A3 (mm), Point: (21, -ro, 0)",-1.873600e-49,-1.579600e-04,-1.204000e-03,-2.811400e-03,-3.208500e-03,-3.364600e-04,6.276400e-03,1.424200e-02,1.878500e-02,1.510600e-02,...,-9.921100e-07,3.778700e-07,1.707300e-06,2.945800e-06,4.047700e-06,4.973300e-06,0.000006,0.000006,0.000006,0.000006
"Radial Displacement A4 (mm), Point: (21, 0, -ro)",-1.873600e-49,-1.579800e-04,-1.204300e-03,-2.812200e-03,-3.209500e-03,-3.360600e-04,6.280500e-03,1.425100e-02,1.879600e-02,1.511200e-02,...,-4.097600e-06,-2.268700e-06,-3.724100e-07,1.516000e-06,3.322700e-06,4.977900e-06,0.000006,0.000008,0.000008,0.000009
"Radial Displacement A1.5 (mm), Point: (21, 0.5*ro*sqrt(2), 0.5*ro*sqrt(2))",-1.846400e-49,-1.566700e-04,-1.199300e-03,-2.804400e-03,-3.203100e-03,-3.831500e-04,6.264600e-03,1.420900e-02,1.874200e-02,1.508400e-02,...,1.675100e-06,9.838500e-06,1.890600e-05,2.720300e-05,3.434000e-05,4.000600e-05,0.000044,0.000046,0.000046,0.000044
"Radial Displacement A2.5 (mm), Point: (21, -0.5*ro*sqrt(2), 0.5*ro*sqrt(2))",-1.845400e-49,-1.569800e-04,-1.201600e-03,-2.808900e-03,-3.207400e-03,-3.822500e-04,6.275200e-03,1.423100e-02,1.877100e-02,1.510600e-02,...,-7.756400e-06,-6.934300e-06,-1.669500e-05,2.668700e-05,3.570200e-05,4.325600e-05,0.000049,0.000053,0.000054,0.000053
"Radial Displacement A3.5 (mm), Point: (21, -0.5*ro*sqrt(2), -0.5*ro*sqrt(2))",-1.849500e-49,-1.550400e-04,-1.183400e-03,-2.765700e-03,-3.164900e-03,-3.895300e-04,6.138900e-03,1.397900e-02,1.849400e-02,1.496600e-02,...,-1.212100e-05,-4.409900e-06,-8.814800e-06,-1.764900e-05,2.609200e-05,3.351100e-05,0.000040,0.000044,0.000046,0.000047
"Radial Displacement A4.5 (mm), Point: (21, 0.5*ro*sqrt(2), -0.5*ro*sqrt(2))",-1.849400e-49,-1.555000e-04,-1.186600e-03,-2.771500e-03,-3.168000e-03,-3.822100e-04,6.160800e-03,1.401000e-02,1.851800e-02,1.495800e-02,...,-9.457600e-06,-5.337000e-06,-9.404500e-06,1.608200e-05,2.256500e-05,2.822200e-05,0.000033,0.000036,0.000037,0.000037
"Radial Displacement A'1 (mm), Point: (420, ro, 0)",2.929000e-78,3.367100e-35,1.547800e-31,4.708100e-29,5.277900e-27,3.447900e-25,1.545000e-23,5.060100e-22,1.259100e-20,2.454000e-19,...,-2.562200e-06,-1.318400e-06,-7.071100e-08,1.130700e-06,2.237500e-06,3.204900e-06,0.000004,0.000005,0.000005,0.000005
"Radial Displacement A'2 (mm), Point: (420, 0, ro)",2.985800e-78,2.342200e-34,2.161500e-31,5.172100e-29,5.514600e-27,3.560100e-25,1.590300e-23,5.203400e-22,1.294000e-20,2.520500e-19,...,-5.949000e-06,-3.538900e-06,-1.054900e-06,1.404100e-06,3.740600e-06,5.862100e-06,0.000008,0.000009,0.000010,0.000011


In [111]:
strict_data.to_csv('I:\\My Drive\\USGW Research\\COMSOL Models\\Pipe 4\\Pipe 4\\d33 Displacements\\Viscoelastic\\Outside Circumference\\Short Axis Length\\2 mm Thickness\\Perpendicular Orientation\\' + sheetname + ' Strict Interval.csv')